# Lab: Large Multimodal Models
**Diplomado en Inteligencia Artificial - Universidad de Chile**

**Duración:** 1.5 horas  
**Modelos:** SmolVLM-500M-Instruct + BLIP-2-OPT-2.7B  
**GPU:** T4 (Google Colab)

---

## Objetivos del Lab
1. Explorar capacidades de modelos multimodales modernos
2. Experimentar con prompting y Visual Question Answering (VQA)
3. Identificar limitaciones sistemáticas de los LMMs
4. Comparar arquitecturas distintas (SmolVLM 2024 vs BLIP-2 2023)

---

## Setup: Instalación y configuración

In [ ]:
"""Instalar dependencias necesarias"""
!pip install -q transformers accelerate pillow torch requests open-clip-torch einops-exts

In [ ]:
"""Imports generales"""
import torch
from PIL import Image
import requests
from io import BytesIO
from transformers import AutoProcessor, AutoModelForVision2Seq
from IPython.display import display

# Verificar GPU disponible
print(f"GPU disponible: {torch.cuda.is_available()}")
print(f"GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU'}")

## Imágenes del Lab

In [ ]:
"""Diccionario de imágenes para el lab"""
IMAGES = {
    # Warm-up: Captioning básico
    "dog_park": "https://irondoggy.com/cdn/shop/articles/dog-running-in-a-dog-park_1258x.jpg?v=1706177184",
    "mountain_lake": "https://storage.googleapis.com/chile-travel-cdn/2021/03/puerto-octay-1024x540-4.jpeg",
    "cooking_kitchen": "https://img.freepik.com/foto-gratis/17-estilos-vida-personas-que-piden-sushi-domicilio_52683-100626.jpg?semt=ais_incoming&w=740&q=80",
    
    # Prompting dirigido
    "urban_scene": "https://dynamic-media-cdn.tripadvisor.com/media/photo-o/03/4f/aa/d8/paseo-ahumada.jpg?w=900&h=500&s=1",
    
    # VQA
    "family_picnic": "https://wallpapers.com/images/hd/the-office-season-8-picnic-y30cb9o08up4apf6.jpg",
    "modern_office": "https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTwkDaHQHdS80lL7I0b2vqvoz2nXnuKf5vmWw&s",
    
    # Límites: Conteo
    "fruit_bowl": "https://magpiestyle.co.nz/cdn/shop/files/IMG_4250_raguwx.jpg?v=1732496459&width=1149",
    
    # Límites: Texto
    "street_sign": "https://ecopsa.com/wp-content/uploads/2015/08/senalizacion-ecopsa2.jpg",
    
    # Límites: Espacial
    "living_room": "https://anticostudio.com/cdn/shop/files/fgf.jpg?v=1749473042",
    
    # Límites: Detalles
    "colorful_objects": "https://dynamic-media-cdn.tripadvisor.com/media/photo-o/16/1f/25/0b/img-20190116-wa0010-01.jpg?w=500&h=500&s=1",
    
    # Memes (opcional)
    "blursed_1": "https://preview.redd.it/blursed-captcha-v0-chc7r4fokz1g1.jpeg?width=1080&crop=smart&auto=webp&s=7fff0b090cfc4cd429252fdcb8f7a27f39b8ce68",
    "blursed_2": "https://preview.redd.it/blursed-bread-v0-eh4bviemlz1g1.jpeg?width=640&crop=smart&auto=webp&s=bf8748fcc5ed8dcf4c68b84bb64a6ebe60034171",
    "text_meme": "https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcS81kLrsNo0aM2miN4Kwys3Mhxqpk8PZHTGGg&s",
}

## Funciones auxiliares

In [ ]:
"""Funciones para cargar imágenes e inferencia"""

def load_image(url):
    """Cargar imagen desde URL"""
    try:
        resp = requests.get(url, timeout=10)
        resp.raise_for_status()
        return Image.open(BytesIO(resp.content)).convert("RGB")
    except Exception as e:
        print(f"❌ Error cargando imagen: {e}")
        return None

def infer_caption_smolvlm(model, processor, image, prompt="", max_tokens=100):
    """Generar texto desde SmolVLM (requiere token <image>)"""
    if image is None:
        return "Error: imagen no válida"
    
    # SmolVLM requiere formato especial con <image> token
    if prompt and "<image>" not in prompt:
        full_prompt = f"<image>{prompt}"
    elif not prompt:
        full_prompt = "<image>Describe this image."
    else:
        full_prompt = prompt
    
    inputs = processor(images=image, text=full_prompt, return_tensors="pt").to(model.device)
    
    with torch.no_grad():
        out = model.generate(**inputs, max_new_tokens=max_tokens)
    
    return processor.decode(out[0], skip_special_tokens=True)

def infer_caption_blip2(model, processor, image, prompt="", max_tokens=100):
    """Generar texto desde BLIP-2 (NO requiere token <image>)"""
    if image is None:
        return "Error: imagen no válida"
    
    inputs = processor(images=image, text=prompt, return_tensors="pt").to(model.device)
    
    with torch.no_grad():
        out = model.generate(**inputs, max_new_tokens=max_tokens)
    
    return processor.decode(out[0], skip_special_tokens=True)

def show_result(image, prompt, result):
    """Mostrar imagen y resultado"""
    print(f"📝 Prompt: {prompt}")
    display(image.resize((400, 300)) if image else None)
    print(f"🤖 Respuesta: {result}\n")
    print("-" * 80)

---
# PARTE 1: SmolVLM-500M-Instruct
Modelo moderno y ligero (~2GB VRAM, 2024)

---

## Cargar modelo SmolVLM

In [ ]:
"""Cargar SmolVLM-500M-Instruct"""
print("Cargando SmolVLM-500M-Instruct...")

MODEL_NAME_1 = "HuggingFaceTB/SmolVLM-500M-Instruct"
processor_1 = AutoProcessor.from_pretrained(MODEL_NAME_1, trust_remote_code=True)
model_1 = AutoModelForVision2Seq.from_pretrained(
    MODEL_NAME_1,
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True
)

print(f"✅ Modelo cargado en: {model_1.device}")
print(f"📊 Parámetros: ~500M")

---
## 1. Warm-up: Captioning básico

Generación automática de descripciones sin instrucciones específicas.

In [ ]:
"""Captioning básico: 3 imágenes simples"""

warmup_images = ["dog_park", "mountain_lake", "cooking_kitchen"]

print("🔥 WARM-UP: Captioning básico\n")
print("=" * 80)

for img_key in warmup_images:
    url = IMAGES[img_key]
    if not url:
        print(f"⚠️  Falta URL para: {img_key}")
        continue
    
    image = load_image(url)
    if image:
        result = infer_caption_smolvlm(model_1, processor_1, image, prompt="Describe this image.")
        show_result(image, "Describe this image.", result)

### 💭 Reflexión
- ¿Las descripciones capturan lo esencial de cada imagen?
- ¿Qué detalles menciona y cuáles omite?
- ¿Hay alucinaciones (menciona cosas que no están)?

---
## 2. Prompting dirigido

Misma imagen, distintos prompts → distintas respuestas.

**Tu tarea:** Completa la lista de prompts con tus propias instrucciones.

In [ ]:
"""Prompting dirigido: experimentar con distintas instrucciones"""

# COMPLETAR: Escribe 3 prompts distintos
prompts = [
    "Describe this image in one sentence.",  # Ejemplo base
    "recreate a history based on this image",  # TU PROMPT 1
    "tell me whats wrong in this image",  # TU PROMPT 2
]

# Cargar imagen
url = IMAGES["urban_scene"]
if url:
    image = load_image(url)
    
    if image:
        print("🎯 PROMPTING DIRIGIDO: Misma imagen, distintos prompts\n")
        print("=" * 80)
        
        for i, prompt in enumerate(prompts, 1):
            if not prompt:
                print(f"⚠️  Prompt {i} vacío - complétalo arriba\n")
                continue
            
            result = infer_caption_smolvlm(model_1, processor_1, image, prompt=prompt)
            show_result(image, prompt, result)
else:
    print("⚠️  Falta URL para 'urban_scene'")

### 💭 Reflexión
- ¿Cómo cambia la respuesta según el prompt?
- ¿Qué tipo de prompt genera mejores resultados?
- ¿Algún prompt confundió al modelo?

---
## 3. Visual Question Answering (VQA)

Hacer preguntas específicas sobre imágenes.

**Tipos de preguntas:**
- **Factual:** Hechos verificables (ej: colores, cantidad)
- **Inferencial:** Requiere interpretación (ej: emociones, intenciones)
- **Razonamiento:** Requiere lógica (ej: por qué, qué pasó antes/después)

**Tu tarea:** Completa las listas de preguntas.

In [ ]:
"""VQA: Preguntas sobre familia en picnic"""

# COMPLETAR: Escribe 3 preguntas (una de cada tipo)
questions_picnic = [
    "How many people are in this image?",  # Ejemplo: factual
    "what emotion are they feeling",  # TU PREGUNTA INFERENCIAL
    "why might they be outdoors",  # TU PREGUNTA DE RAZONAMIENTO
]

url = IMAGES["family_picnic"]
if url:
    image = load_image(url)
    
    if image:
        print("❓ VQA: Familia en picnic\n")
        print("=" * 80)
        
        for q in questions_picnic:
            if not q:
                print("⚠️  Pregunta vacía - complétala arriba\n")
                continue
            
            result = infer_caption_smolvlm(model_1, processor_1, image, prompt=q)
            show_result(image, q, result)
else:
    print("⚠️  Falta URL para 'family_picnic'")

In [ ]:
"""VQA: Preguntas sobre oficina moderna"""

# COMPLETAR: Escribe 3 preguntas (una de cada tipo)
questions_office = [
    "What technology can you see in this image?",  # Ejemplo: factual
    "what type of service do they provide",  # TU PREGUNTA INFERENCIAL
    "how much do they make based on the resources displayed in the image?",  # TU PREGUNTA DE RAZONAMIENTO
]

url = IMAGES["modern_office"]
if url:
    image = load_image(url)
    
    if image:
        print("❓ VQA: Oficina moderna\n")
        print("=" * 80)
        
        for q in questions_office:
            if not q:
                print("⚠️  Pregunta vacía - complétala arriba\n")
                continue
            
            result = infer_caption_smolvlm(model_1, processor_1, image, prompt=q)
            show_result(image, q, result)
else:
    print("⚠️  Falta URL para 'modern_office'")

### 💭 Reflexión
- ¿El modelo responde correctamente las preguntas factuales?
- ¿Puede hacer inferencias razonables?
- ¿Dónde es más débil: hechos, inferencias, o razonamiento?

---
## 4. Encontrando los límites

Probar sistemáticamente dónde fallan los modelos multimodales.

### a) Conteo de objetos

In [ ]:
"""Límite 1: Conteo de objetos"""

url = IMAGES["fruit_bowl"]
if url:
    image = load_image(url)
    
    if image:
        print("🔢 LÍMITE: Conteo de objetos\n")
        print("=" * 80)
        
        # Preguntas de conteo
        count_questions = [
            "How many fruits are in this image?",
            "Count all the objects you can see.",
            "How many red objects are there?"
        ]
        
        for q in count_questions:
            result = infer_caption_smolvlm(model_1, processor_1, image, prompt=q)
            show_result(image, q, result)
else:
    print("⚠️  Falta URL para 'fruit_bowl'")

**Observación:** Los LMMs generalmente son malos contando. ¿Acertó en este caso?

### b) Texto en imágenes (lectura OCR)

In [ ]:
"""Límite 2: Leer texto en imágenes"""

url = IMAGES["street_sign"]
if url:
    image = load_image(url)
    
    if image:
        print("📝 LÍMITE: Texto en imágenes\n")
        print("=" * 80)
        
        text_questions = [
            "What does the sign say?",
            "Read the text in this image.",
            "What words can you see?"
        ]
        
        for q in text_questions:
            result = infer_caption_smolvlm(model_1, processor_1, image, prompt=q)
            show_result(image, q, result)
else:
    print("⚠️  Falta URL para 'street_sign'")

**Observación:** ¿Puede leer correctamente? ¿Solo parcialmente? ¿Alucina texto?

### c) Razonamiento espacial

In [ ]:
"""Límite 3: Razonamiento espacial (posiciones relativas)"""

url = IMAGES["living_room"]
if url:
    image = load_image(url)
    
    if image:
        print("📐 LÍMITE: Razonamiento espacial\n")
        print("=" * 80)
        
        spatial_questions = [
            "What is to the left of the sofa?",
            "Describe the position of objects in the room.",
            "What is in the center of the image?"
        ]
        
        for q in spatial_questions:
            result = infer_caption_smolvlm(model_1, processor_1, image, prompt=q)
            show_result(image, q, result)
else:
    print("⚠️  Falta URL para 'living_room'")

**Observación:** ¿Entiende correctamente izquierda/derecha, arriba/abajo?

### d) Detalles finos

In [ ]:
"""Límite 4: Detalles finos (colores, tamaños, texturas)"""

url = IMAGES["colorful_objects"]
if url:
    image = load_image(url)
    
    if image:
        print("🔍 LÍMITE: Detalles finos\n")
        print("=" * 80)
        
        detail_questions = [
            "What color is the smallest object?",
            "Describe the textures you can see.",
            "What is the brightest colored item?"
        ]
        
        for q in detail_questions:
            result = infer_caption_smolvlm(model_1, processor_1, image, prompt=q)
            show_result(image, q, result)
else:
    print("⚠️  Falta URL para 'colorful_objects'")

**Observación:** ¿Nota detalles pequeños o solo lo más obvio?

### 💭 Resumen de límites de SmolVLM

Escribe 3-4 oraciones resumiendo:
- ¿En qué tareas el modelo es bueno?
- ¿Dónde falla consistentemente?
- ¿Qué tipo de errores comete (alucinaciones, imprecisión, etc)?

**Tu análisis:**

*(Escribe aquí)*

---
# PARTE 2: BLIP-2-OPT-2.7B
Modelo de 2023, arquitectura clásica (~2.7B parámetros, ~5GB VRAM)

Vamos a repetir algunos experimentos clave para comparar con SmolVLM.

---

## Liberar memoria y cargar BLIP-2

In [ ]:
"""Liberar VRAM del modelo anterior"""
print("🧹 Liberando memoria...")

del model_1, processor_1
torch.cuda.empty_cache()

print(f"✅ Memoria liberada. VRAM disponible: ~{torch.cuda.mem_get_info()[0] / 1024**3:.1f} GB")

In [ ]:
"""Cargar BLIP-2-OPT-2.7B"""
print("Cargando BLIP-2-OPT-2.7B...")

from transformers import Blip2Processor, Blip2ForConditionalGeneration

MODEL_NAME_2 = "Salesforce/blip2-opt-2.7b"
processor_2 = Blip2Processor.from_pretrained(MODEL_NAME_2)
model_2 = Blip2ForConditionalGeneration.from_pretrained(
    MODEL_NAME_2,
    torch_dtype=torch.float16,
    device_map="auto"
)

print(f"✅ Modelo cargado en: {model_2.device}")
print(f"📊 Parámetros: ~2.7B")
print(f"📅 Año: 2023 (arquitectura clásica)")

---
## Comparación 1: Prompting dirigido

Repetimos el experimento de prompting con la misma imagen y prompts.

In [ ]:
"""Comparación: Prompting con BLIP-2"""

# Usar los mismos prompts que antes
prompts_comparison = [
    "Describe this image in one sentence.",
    "List all the objects and people you can see.",
    "What is the overall mood or atmosphere of this scene?"
]

url = IMAGES["urban_scene"]
if url:
    image = load_image(url)
    
    if image:
        print("🔄 COMPARACIÓN: Prompting (BLIP-2)\n")
        print("=" * 80)
        
        for prompt in prompts_comparison:
            result = infer_caption_blip2(model_2, processor_2, image, prompt=prompt)
            show_result(image, prompt, result)
else:
    print("⚠️  Falta URL para 'urban_scene'")

### 💭 Comparación SmolVLM vs BLIP-2 (Prompting)

- ¿Cuál dio descripciones más detalladas?
- ¿Cuál fue más preciso?
- ¿Notaste diferencias en el estilo de respuesta?
- ¿El modelo más nuevo (SmolVLM 2024) es realmente mejor que BLIP-2 (2023)?

**Tu análisis:**

*(Escribe aquí)*

---
## Comparación 2: Límites (Conteo + Texto)

In [ ]:
"""Comparación: Conteo con BLIP-2"""

url = IMAGES["fruit_bowl"]
if url:
    image = load_image(url)
    
    if image:
        print("🔢 COMPARACIÓN: Conteo (BLIP-2)\n")
        print("=" * 80)
        
        q = "How many fruits are in this image?"
        result = infer_caption_blip2(model_2, processor_2, image, prompt=q)
        show_result(image, q, result)
else:
    print("⚠️  Falta URL para 'fruit_bowl'")

In [ ]:
"""Comparación: Texto con BLIP-2"""

url = IMAGES["street_sign"]
if url:
    image = load_image(url)
    
    if image:
        print("📝 COMPARACIÓN: Texto (BLIP-2)\n")
        print("=" * 80)
        
        q = "What does the sign say?"
        result = infer_caption_blip2(model_2, processor_2, image, prompt=q)
        show_result(image, q, result)
else:
    print("⚠️  Falta URL para 'street_sign'")

### 💭 Comparación SmolVLM vs BLIP-2 (Límites)

- ¿BLIP-2 es mejor en conteo?
- ¿BLIP-2 lee mejor el texto?
- ¿Vale la pena usar el modelo más nuevo (SmolVLM) o el clásico (BLIP-2) es suficiente?
- ¿Qué limitaciones comparten ambos modelos?

**Tu análisis:**

*(Escribe aquí)*

---
## Bonus: Memes y bromas con BLIP-2

Probemos los límites del modelo con imágenes "raras".

In [ ]:
"""Bonus: Explicar memes/imágenes confusas con BLIP-2"""

meme_keys = ["blursed_1", "text_meme"]

print("🎭 BONUS: Memes y contenido confuso (BLIP-2)\n")
print("=" * 80)

for meme_key in meme_keys:
    url = IMAGES.get(meme_key)
    if not url:
        print(f"⚠️  Falta URL para: {meme_key}")
        continue
    
    image = load_image(url)
    if image:
        # Intentar explicar qué es raro/gracioso
        prompts_meme = [
            "Describe what you see in this image.",
            "What is unusual or funny about this image?",
            "Explain why this might be considered humorous."
        ]
        
        for p in prompts_meme:
            result = infer_caption_blip2(model_2, processor_2, image, prompt=p)
            show_result(image, p, result)
        
        print("\n" + "=" * 80 + "\n")

### 💭 Reflexión sobre memes

- ¿BLIP-2 captó el humor o la rareza?
- ¿Qué tipo de conocimiento necesitaría para "entender" la broma?
- ¿Es esto una limitación fundamental o solo cuestión de entrenamiento?

**Tu análisis:**

*(Escribe aquí)*

---
# PARTE 3: Playground libre con SmolVLM

Volvemos a cargar SmolVLM para experimentar libremente.

---

## Re-cargar SmolVLM para playground

In [ ]:
"""Liberar VRAM de BLIP-2"""
print("🧹 Liberando memoria de BLIP-2...")

del model_2, processor_2
torch.cuda.empty_cache()

print(f"✅ Memoria liberada. VRAM disponible: ~{torch.cuda.mem_get_info()[0] / 1024**3:.1f} GB")

In [ ]:
"""Re-cargar SmolVLM-500M-Instruct para playground"""
print("Recargando SmolVLM-500M-Instruct para playground...")

processor_playground = AutoProcessor.from_pretrained("HuggingFaceTB/SmolVLM-500M-Instruct", trust_remote_code=True)
model_playground = AutoModelForVision2Seq.from_pretrained(
    "HuggingFaceTB/SmolVLM-500M-Instruct",
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True
)

print(f"✅ SmolVLM recargado para experimentación libre")

## Función para probar imágenes propias

In [ ]:
"""Función para probar imágenes propias"""

def probar_imagen(url, prompt="Describe this image in detail."):
    """
    Prueba el modelo con tu propia imagen.
    
    Args:
        url: URL de la imagen (Unsplash, Reddit, etc)
        prompt: Pregunta o instrucción para el modelo
    
    Returns:
        None (muestra imagen y resultado)
    """
    image = load_image(url)
    if image:
        result = infer_caption_smolvlm(model_playground, processor_playground, image, prompt=prompt, max_tokens=150)
        show_result(image, prompt, result)
    else:
        print("❌ Error cargando imagen. Verifica la URL.")

## Experimenta aquí

Prueba con tus propias imágenes. Algunas ideas:
- Foto personal
- Imagen de tu ciudad
- Algo específico de tu trabajo/investigación
- Imagen técnica (diagrama, gráfico, etc)
- Memes o imágenes raras

In [ ]:
"""EXPERIMENTA AQUÍ"""

# Ejemplo 1: Tu imagen
probar_imagen(
    url="",  # TU URL AQUÍ
    prompt="Describe this image."  # TU PROMPT AQUÍ
)

In [ ]:
# Ejemplo 2: Otra imagen
probar_imagen(
    url="",  # TU URL AQUÍ
    prompt=""  # TU PROMPT AQUÍ
)

In [ ]:
# Ejemplo 3: Más experimentos
probar_imagen(
    url="",  # TU URL AQUÍ
    prompt=""  # TU PROMPT AQUÍ
)

---
# Conclusiones del Lab

## Resumen de aprendizajes:

1. **Capacidades básicas:**
   - *(¿Qué hacen bien los LMMs en general?)*

2. **Limitaciones identificadas:**
   - *(¿Dónde fallan sistemáticamente? Conteo, texto, espacial, etc)*

3. **Diferencias entre modelos:**
   - *(SmolVLM 2024 vs BLIP-2 2023: ¿ha mejorado realmente la tecnología?)*
   - *(¿Cuándo vale la pena usar un modelo más grande/nuevo?)*

4. **Implicaciones prácticas:**
   - *(¿Para qué casos de uso sí/no usarías estos modelos?)*
   - *(¿Qué tipo de aplicaciones son viables hoy?)*

**Tus conclusiones:**

*(Escribe aquí)*

---
## Recursos adicionales

### Modelos:
- **SmolVLM:** https://huggingface.co/HuggingFaceTB/SmolVLM-500M-Instruct
- **BLIP-2:** https://huggingface.co/Salesforce/blip2-opt-2.7b
- **Otros LMMs:**
  - LLaVA: https://huggingface.co/llava-hf
  - Qwen2-VL: https://huggingface.co/Qwen
  - PaliGemma: https://huggingface.co/google/paligemma

### Papers:
- **BLIP-2:** https://arxiv.org/abs/2301.12597
- **CLIP:** https://arxiv.org/abs/2103.00020
- **Flamingo:** https://arxiv.org/abs/2204.14198
- **LLaVA:** https://arxiv.org/abs/2304.08485

### Datasets de evaluación:
- **VQAv2:** https://visualqa.org/
- **COCO Captions:** https://cocodataset.org/
- **TextVQA:** https://textvqa.org/
- **GQA:** https://cs.stanford.edu/people/dorarad/gqa/

### Tutoriales:
- **Hugging Face Vision:** https://huggingface.co/docs/transformers/tasks/image_captioning
- **OpenAI Vision Guide:** https://platform.openai.com/docs/guides/vision

---
**Fin del Lab**